### Import data

In [1]:
import os
import pandas as pd

DATASET_PATH = '/home/alk/aau/datasets'

SVEBOLLE_PATH = os.path.join(DATASET_PATH, 'Svebolle')
SVEBOLLE_CSV = 'LORA_data.csv'



def load_data(path, csv_file, sep=','):
    """default sep param is ','"""
    csv_path = os.path.join(path, csv_file)
    return pd.read_csv(csv_path, sep)


data = load_data(SVEBOLLE_PATH, SVEBOLLE_CSV,
                 sep=';')

### Creating time-series for activity in time (resolution = seconds)

In [2]:
data = data[data['mType']=='Confirmed Data Up']
data = data[['Time', 'DevAddr']]
data.head()

,Time,DevAddr
0,2017-01-02 12:08:27.788000,000013c1
1,2017-01-02 12:08:29.196000,000013b8
2,2017-01-02 12:08:44.520000,000013bf
3,2017-01-02 12:08:55.347000,000013bf
4,2017-01-02 12:08:55.346000,000013bf


In [3]:
# removing nanoseconds
Time = list(data.Time.values)
Time_parsed = []
sep = '.'
for t in Time:
    t_parsed = t.split(sep, 1)[0]
    Time_parsed.append(t_parsed)

In [4]:
Time_parsed

['2017-01-02 12:08:27',
 '2017-01-02 12:08:29',
 '2017-01-02 12:08:44',
 '2017-01-02 12:08:55',
 '2017-01-02 12:08:55',
 '2017-01-02 12:09:00',
 '2017-01-02 12:09:04',
 '2017-01-02 12:09:09',
 '2017-01-02 12:09:29',
 '2017-01-02 12:09:44',
 '2017-01-02 12:10:29',
 '2017-01-02 12:10:35',
 '2017-01-02 12:10:44',
 '2017-01-02 12:11:09',
 '2017-01-02 12:11:14',
 '2017-01-02 12:11:18',
 '2017-01-02 12:11:29',
 '2017-01-02 12:11:49',
 '2017-01-02 12:12:27',
 '2017-01-02 12:12:42',
 '2017-01-02 12:12:44',
 '2017-01-02 12:13:12',
 '2017-01-02 12:13:18',
 '2017-01-02 12:13:23',
 '2017-01-02 12:13:27',
 '2017-01-02 12:13:29',
 '2017-01-02 12:14:26',
 '2017-01-02 12:14:44',
 '2017-01-02 12:15:28',
 '2017-01-02 12:15:28',
 '2017-01-02 12:15:29',
 '2017-01-02 12:15:29',
 '2017-01-02 12:15:32',
 '2017-01-02 12:15:36',
 '2017-01-02 12:16:27',
 '2017-01-02 12:16:44',
 '2017-01-02 12:16:54',
 '2017-01-02 12:16:54',
 '2017-01-02 12:17:29',
 '2017-01-02 12:17:29',
 '2017-01-02 12:17:36',
 '2017-01-02 12:

In [5]:
data.Time = Time_parsed
data['Time'] = pd.to_datetime(data['Time'])
data.head()

,Time,DevAddr
0,2017-01-02 12:08:27,000013c1
1,2017-01-02 12:08:29,000013b8
2,2017-01-02 12:08:44,000013bf
3,2017-01-02 12:08:55,000013bf
4,2017-01-02 12:08:55,000013bf


In [6]:
# sorting ascending
data = data.sort_values('Time')
data = data.reset_index(drop=True)
print('number of transmissions:', len(data))
data.head()

number of transmissions: 689123


,Time,DevAddr
0,2017-01-02 00:07:00,000013b7
1,2017-01-02 02:40:00,000013bf
2,2017-01-02 02:51:00,000013c1
3,2017-01-02 12:08:27,000013c1
4,2017-01-02 12:08:29,000013b8


### Meta time-series for activity of every device in time

In [7]:
columns = list(data.DevAddr.unique())

In [8]:
datetime_index = pd.DatetimeIndex(start='2017-08-01 00:00',freq='s', end='2017-08-01 23:59:59')
len(datetime_index)

86400

In [9]:
import numpy as np

d = {c : np.zeros(shape=(len(datetime_index), )) for c in columns}

In [10]:
df_temp = pd.DataFrame(data={'Time' : datetime_index,
                             'dev' : np.zeros(shape=(len(datetime_index), )) }
                      )
df_temp = df_temp.set_index('Time')
df_temp.head()

,dev
Time,
2017-08-01 00:00:00,0.0
2017-08-01 00:00:01,0.0
2017-08-01 00:00:02,0.0
2017-08-01 00:00:03,0.0
2017-08-01 00:00:04,0.0


In [11]:
# take for example device 13b7
data_temp = data[data['DevAddr'] == '000013b7']
data_temp = data_temp.set_index('Time')
data_temp.head()

,DevAddr
Time,
2017-01-02 00:07:00,000013b7
2017-01-02 12:09:00,000013b7
2017-01-02 12:09:04,000013b7
2017-01-02 12:09:09,000013b7
2017-01-02 12:11:09,000013b7


In [ ]:
# new dataset filled w/ ones for every time device 13b7 was active
ind = list(data_temp.index.values)
for i in ind:
    df_temp.loc[i, 'dev'] = 1.0

### Linear model on time series

#### Feature extraction
Since the model requires features and df_temp is nothin but 1D time series, features can be:
1. lags of time series
2. ~~window statistics~~
3. date and time features (is it weekday, weekend, holiday, special day?)
4. target encoding
5. forecast from other models

In [ ]:
df_temp.head()

In [ ]:
data_for_lr = pd.DataFrame(df_temp.dev.copy())
data_for_lr.columns = ['y']

In [ ]:
for i in range (0, 10):
    data_for_lr['lag_{}'.format(i)] = data_for_lr.y.shift(i)

In [ ]:
data_for_lr.head()

In [ ]:
data_for_lr.tail()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
def timeseries_train_test_split(X, y, test_size):
    """ performing train-test split with respect to time series structure """
    
    test_index = int(len(X) * (1 - test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [ ]:
y = data_for_lr.dropna().y
X = data_for_lr.dropna().drop(['y'], axis=1)

X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, 0.3)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
from sklearn.externals import joblib
filename = 'linreg_model_no_regularization.pkl'
_ = joblib.dump(lr, filename)

In [ ]:
def plotModelRes(model, X_train=X_train, X_test=X_test,
                 plot_intervals=False,
                 plot_anomalies=False):
    """Modeled v fact values"""
    prediction = model.predict(X_test)
    
    plt.figure(figsize=(12, 6))
    plt.plot(prediction, 'g', label='prediction')
    plt.plot(y_test.values, label='actual')
    
    if plot_intervals:
        cv = cross_val_score(model, X_train, y_train,
                                cv=tscv,
                                scoring='neg_mean_absolute_error')
        mae = cv.mean() * (-1)
        deviation = cv.std()
        
        scale = 1.96
        lower = prediction - (mae + scale*deviation)
        upper = prediction + (mae + scale*deviation)
        
        plt.plot(lower, 'r--', label='upper/lower bond', alpha=0.5)
        plt.plot(upper, 'r--', alpha=0.5)
        
    if plot_anomalies:
        anomalies = np.array([np.NaN]*len(y_test))
        anomalies[y_test<lower] = y_test[y_test<lower]
        anomalies[y_test>upper] = y_test[y_test>upper]
        plt.plot(anomalies, 'o', markersize=10, label='anomalies')
    
    error = mean_absolute_percentage_error(prediction, y_test)
    plt.title('Mean Absolute Percentage Error {0:.2f}%'.format(error))
    plt.legend(loc='best')
    plt.tight_layout()
    plt.grid(True)
    
def plotCoeffs(model):
    """Sorted Coefficient values of the model"""
    coefs = pd.DataFrame(model.coef_, X_train.columns)
    coefs.columns = ['coef']
    coefs['abs'] = coefs.coef.apply(np.abs)
    coefs = coefs.sort_values(by='abs', ascending=False).drop(['abs'], axis=1)
    
    plt.figure(figsize=(15, 7))
    coefs.coef.plot(kind='bar')
    plt.grid(True, axis='y')
    plt.hlines(y=0, xmin=0, xmax=len(coefs), linestyles='dashed');


In [ ]:
prediction = lr.predict(X_test)

In [ ]:
data.index = pd.to_datetime(data.index)
data['weekday'] = data.index.weekday
data.tail()

In [ ]:
plt.figure(figsize=(12, 6))
plt.title('encoded featres')
data.iloc[1200:].weekday.plot()
plt.grid(True)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [ ]:
y = data.dropna().y
X = data.dropna().drop(['y'], axis=1)

X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

plotModelRes(lr, X_train=X_train_scaled, X_test=X_test_scaled, plot_intervals=True, plot_anomalies=True)
plotCoeffs(lr)

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV

ridge = RidgeCV(cv=tscv)
ridge.fit(X_train_scaled, y_train)

plotModelRes(ridge, 
                 X_train=X_train_scaled, 
                 X_test=X_test_scaled, 
                 plot_intervals=True, plot_anomalies=True)
plotCoeffs(ridge)